In [9]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="courtlin-udacity",subscription_id="bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8", resource_group="udacity-nano")
exp = Experiment(workspace=ws, name="courtlin-udacity")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: courtlin-udacity
Azure region: eastus2
Subscription id: bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8
Resource group: udacity-nano


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute import ComputeTarget, AmlCompute
cpu_cluster_name = "udacityproj1"
compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5, vm_priority='lowpriority',min_nodes=0, idle_seconds_before_scaledown=300)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice

# ps = RandomParameterSampling( {
#         "learning_rate": normal(10, 3),
#         "keep_probability": uniform(0.05, 0.1),
#         "batch_size": choice(16, 32, 64, 128)
#     }
# )


ps = RandomParameterSampling(
    {
        "--C": choice(0.1,1.0,10.0),
        "--max_iter": choice(5,10,20)
    }
)


# Specify an Early Termination Policy
# policy = ### YOUR CODE HERE ###
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###


from azureml.core import ScriptRunConfig

# import os
# est = SKLearn(source_directory=os.path.join('./'),
#                       entry_script='train.py',
#                       compute_target=cpu_cluster,
#              )

compute_target = ws.compute_targets['udacityproj1']

# from azureml.core import Environment
# sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

# src = ScriptRunConfig(source_directory=os.path.join('./'),
#                       script='train.py',
#                       arguments=['--kernel', 'linear', '--penalty', 1.0],
#                       compute_target=compute_target,
#                       environment=sklearn_env)

# est = SKLearn(source_directory=os.path.join('./'),
#                       entry_script='train.py',
#                       compute_target=cpu_cluster,
#              )

# Create a SKLearn estimator for use with train.py
# est = SKLearn(
#     source_directory=".",
#     compute_target=compute_target,
#     entry_script="train.py"
# )


# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=os.path.join('./'),
    compute_target=compute_target,
    entry_script="train.py"
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
                             estimator=est,
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=10)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "courtlin-udacity")
hyperdrive_run = experiment.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()



In [ ]:
# from azureml.widgets import RunDetails
# RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
from azureml.core.model import Model

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

# model = best_run.register_model(model_name='hyperdrive_model', model_path='./outputs/model.pkl', model_framework=Model.Framework.SCIKITLEARN, model_framework_version='0.19.1')

### YOUR CODE HERE ###

# best_run = hyperdrive_run.get_best_run_by_primary_metric()
# best_run_metrics = best_run.get_metrics()
# parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
# print('\n learning rate:',parameter_values[3])
# print('\n keep probability:',parameter_values[5])
# print('\n batch size:',parameter_values[7])

In [ ]:
import joblib
best_run, fitted_model = hyperdrive_run.get_output_by_primary_metric()
print(best_run)
print(fitted_model)
joblib.dump(fitted_model, "fitted_hyperdrive_model.joblib")

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

from azureml.core.dataset import Dataset

url_paths = [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
    ]

ds = TabularDatasetFactory.from_delimited_files(path=url_paths)
type(ds)

azureml.data.tabular_dataset.TabularDataset

In [ ]:
# print(ds.to_pandas_dataframe())

In [29]:
from train import clean_data

# Use the clean_data function to clean your data.
# x, y = clean_data(ds)

from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

# TODO: Create TabularDataset using TabularDatasetFactory
# Data is located at:
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

from azureml.core.dataset import Dataset

url_paths = [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
            ]

ds = TabularDatasetFactory.from_delimited_files(path=url_paths)

run = Run.get_context(allow_offline=True)

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)

    return x_df, y_df

x, y = clean_data(ds)

#is this necessary with automl
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=123)

train_data = x.join(y)

train_data.to_csv("data/train_data.csv", index=False)
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)
 
# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
print(train_data.to_pandas_dataframe())
# clean_data = x
# clean_data['y'] = y 


type(train_data)


Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
       age  marital  default  housing  loan  month  day_of_week  duration  \
0       57        1        0        0     1      5            1       371   
1       55        1        0        1     0      5            4       285   
2       33        1        0        0     0      5            5        52   
3       36        1        0        0     0      6            5       355   
4       27        1        0        1     0      7            5       189   
...    ...      ...      ...      ...   ...    ...          ...       ...   
32945   56        1        0        0     1      7            1       116   
32946   37        1        0        0     1      7            5        69   
32947   26        0        0        0     0      5            2       135   
32948   31        0        0        0     0      4            1       386   

azureml.data.tabular_dataset.TabularDataset

In [30]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
# automl_config = AutoMLConfig(
#     experiment_timeout_minutes=30,
#     task=,
#     primary_metric=,
#     training_data=,
#     label_column_name=,
#     n_cross_validations=)

automl_config = AutoMLConfig(
    experiment_timeout_hours=.5,
    task="classification",
    primary_metric="accuracy",
    training_data= train_data,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=compute_target,
    max_concurrent_iterations=5,
    max_cores_per_iteration=2
)

In [31]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)


Submitting remote run.
No run_configuration provided, running on udacityproj1 with default configuration
Running on remote compute: udacityproj1


Experiment,Id,Type,Status,Details Page,Docs Page
courtlin-udacity,AutoML_44de51ab-67a6-4580-9a73-07a3bb219c34,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

In [32]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
# best_run_metrics = best_automl_run.get_metrics() 
# for metric_name in best_run_metrics:
#      metric = best_run_metrics[metric_name]
#      print(metric_name, metric)


best_run, best_model = automl_run.get_output()

print("Best run:")
print(best_run)
print("Best model:")
print(best_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Best run:
Run(Experiment: courtlin-udacity,
Id: AutoML_44de51ab-67a6-4580-9a73-07a3bb219c34_29,
Type: azureml.scriptrun,
Status: Completed)
Best model:
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=0.5208333333333334, subsample=0.6, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.2857142857142857, 0.2857142857142857, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))],
         verbose=False)


In [36]:
best_run_metrics = automl_run.get_metrics() 
for metric_name in best_run_metrics:
     metric = best_run_metrics[metric_name]
     print(metric_name, metric)

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetBalancing', 'DatasetCrossValidationSplit', 'ModelSelection', 'BestRunExplainModel', 'ModelExplanationDataSetSetup', 'PickSurrogateModel', 'EngineeredFeatureExplanations', 'EngineeredFeatureExplanations', 'RawFeaturesExplanations', 'RawFeaturesExplanations', 'BestRunExplainModel']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Performing class balancing sweeping', 'Generating individually featurized CV splits.', 'Beginning model selection.', 'Best run model explanations started', 'Model explanations data setup completed', 'Choosing LightGBM as the surrogate model for explanations', 'Computation of engineered features started', 'Computation of engineered features completed', 'Computation of 

In [37]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [38]:
import joblib
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
joblib.dump(fitted_model, "fitted_automl_model.joblib")

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Run(Experiment: courtlin-udacity,
Id: AutoML_44de51ab-67a6-4580-9a73-07a3bb219c34_29,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=0.5208333333333334, subsample=0.6, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.2857142857142857, 0.2857142857142857, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))],
         verbose=False)


['fitted_automl_model.joblib']

In [ ]:
# Delete compute cluster
cpu_cluster = ComputeTarget.delete(ws, cpu_cluster_name)